In [2]:
!pip install -q --upgrade pip
!pip install -q "transformers>=4.30.0" "datasets" "accelerate>=0.21.0" "peft>=0.4.0" "bitsandbytes>=0.39.0" "sentencepiece" "safetensors"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.5 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
gradio 5.38.1 requires pyda

In [1]:
!pip install transformers accelerate torchinfo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 10.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00:00:0100:01
  Attempting un

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "/kaggle/input/llama-3.2-3b-instruct/pytorch/default/1"   # ← change this to your model

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",      # loads layers automatically on available GPUs/CPU
    torch_dtype="auto"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
from torchinfo import summary
import torch.nn as nn
summary(model, depth=9, col_names=["kernel_size", "num_params", "trainable"])

print("\nDetailed Linear Layers:\n" + "="*40)
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        d, k = module.weight.shape
        print(f"{name:<70}  {d:>5} x {k:<5}  = {d*k:,} params")
        
print("\nTotal trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))


Detailed Linear Layers:
model.layers.0.self_attn.q_proj                                          3072 x 3072   = 9,437,184 params
model.layers.0.self_attn.k_proj                                          1024 x 3072   = 3,145,728 params
model.layers.0.self_attn.v_proj                                          1024 x 3072   = 3,145,728 params
model.layers.0.self_attn.o_proj                                          3072 x 3072   = 9,437,184 params
model.layers.0.mlp.gate_proj                                             8192 x 3072   = 25,165,824 params
model.layers.0.mlp.up_proj                                               8192 x 3072   = 25,165,824 params
model.layers.0.mlp.down_proj                                             3072 x 8192   = 25,165,824 params
model.layers.1.self_attn.q_proj                                          3072 x 3072   = 9,437,184 params
model.layers.1.self_attn.k_proj                                          1024 x 3072   = 3,145,728 params
model.layers.1.sel

In [10]:
import torch
from prettytable import PrettyTable

# Get one transformer block (acts like one encoder/decoder layer)
block = model.model.layers[0]

print("=== TRANSFORMER BLOCK STRUCTURE ===")
print(block)

# Function to recursively list modules and parameter counts
def describe_module(module, prefix=""):
    rows = []
    total_params = 0
    for name, child in module.named_children():
        params = sum(p.numel() for p in child.parameters())
        total_params += params
        rows.append([prefix + name, child.__class__.__name__, f"{params:,}"])
        rows += describe_module(child, prefix + "  ")
    return rows

rows = describe_module(block)

# Pretty print the architecture
table = PrettyTable()
table.field_names = ["Layer Name", "Type", "Parameters"]
for r in rows:
    table.add_row(r)
print(table)


=== TRANSFORMER BLOCK STRUCTURE ===
LlamaDecoderLayer(
  (self_attn): LlamaAttention(
    (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
    (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
    (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
    (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
    (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
    (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
)
+--------------------------+----------------+------------+
|        Layer Name        |      Type      | Parameters |
+--------------------------+----------------+------------+
|        self_attn         | LlamaAttention | 25,165,824 |
|           q_

In [11]:
num_layers = len(model.model.layers)
print(f"Number of decoder layers: {num_layers}")

Number of decoder layers: 28


In [3]:
import os
import sys
import json
import glob
from pathlib import Path
import torch

print("Python:", sys.version.splitlines()[0])
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))
else:
    print("Warning: CUDA not available. QLoRA requires GPU for practical training.")

import transformers, datasets, peft, bitsandbytes, accelerate
print("transformers:", transformers.__version__)
print("datasets:", datasets.__version__)
print("peft:", peft.__version__)
import bitsandbytes as bnb
print("bitsandbytes:", bnb.__version__)

Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Torch: 2.6.0+cu124
CUDA available: True
CUDA device: Tesla T4


2025-10-23 21:09:38.220712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761253778.452965      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761253778.520890      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


transformers: 4.53.3
datasets: 4.1.1
peft: 0.16.0
bitsandbytes: 0.48.1


In [4]:
DATA_PATH = "/kaggle/input/finance-finetuning-dataset-combined"
BASE_MODEL_PATH = "/kaggle/input/llama-3.2-3b-instruct/pytorch/default/1"
OUTPUT_DIR = "/kaggle/working/qlora_lora_finance"  # where adapters/checkpoints will be saved

# Training hyperparameters
MICRO_BATCH_SIZE = 4           
GRAD_ACCUM_STEPS = 8           
EPOCHS = 3
LEARNING_RATE = 2e-4
CUTOFF_LEN = 1024              # token sequence length (common for LLaMA-like models)
LORA_R = 8
LORA_ALPHA = 32
LORA_DROPOUT = 0.05
SEED = 42

os.makedirs(OUTPUT_DIR, exist_ok=True)
print("DATA_PATH:", DATA_PATH)
print("BASE_MODEL_PATH:", BASE_MODEL_PATH)
print("OUTPUT_DIR:", OUTPUT_DIR)


DATA_PATH: /kaggle/input/finance-finetuning-dataset-combined
BASE_MODEL_PATH: /kaggle/input/llama-3.2-3b-instruct/pytorch/default/1
OUTPUT_DIR: /kaggle/working/qlora_lora_finance


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer - prefer local model path; fallback to hub if needed
print("Loading tokenizer from:", BASE_MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, use_fast=False, trust_remote_code=True)
# Some LLaMA tokenizers do not have pad_token; set it to eos_token for causal LM training
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# BitsAndBytes 4-bit config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

print("Loading base model (4-bit) — this can take a while.")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
)

# Freeze base model parameters
for param in model.parameters():
    param.requires_grad = False

print("Model loaded. Trainable params initially:", sum(p.numel() for p in model.parameters() if p.requires_grad))


Loading tokenizer from: /kaggle/input/llama-3.2-3b-instruct/pytorch/default/1
Loading base model (4-bit) — this can take a while.


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded. Trainable params initially: 0


In [6]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Prepare the model for k-bit training (adjusts some layers / casts)
model = prepare_model_for_kbit_training(model)

# PEFT will ignore them. Typical attention projection names for LLaMA-like models:
target_modules = [
    "q_proj", "v_proj", "k_proj", "o_proj",
    "gate_proj", "down_proj", "up_proj",
    "wi", "wo", "wq", "wv", "wk", "wo"
]

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=target_modules,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # show how many params will be trained


trainable params: 12,156,928 || all params: 3,224,906,752 || trainable%: 0.3770


In [7]:
from datasets import load_dataset, Dataset, DatasetDict

def find_data_files(path):
    p = Path(path)
    if p.is_file():
        return [str(p)]
    # If it's a dir, look for .jsonl/.json/.csv
    files = []
    for ext in ("*.jsonl","*.json","*.csv"):
        files.extend(sorted([str(x) for x in p.glob(ext)]))
    return files

data_files = find_data_files(DATA_PATH)
if len(data_files) == 0:
    raise FileNotFoundError(f"No dataset files found at {DATA_PATH}. Expect .jsonl/.json/.csv or a file path.")
print("Found data files:", data_files[:10])

# The dataset has fields: instruction, input, output (answer)
sample_file = data_files[0]
if sample_file.endswith(".csv"):
    raw = load_dataset("csv", data_files=data_files)
else:
    # try json (datasets handles jsonl too)
    raw = load_dataset("json", data_files=data_files)

# If the dataset has splits, unify to train split
if "train" in raw:
    ds = raw["train"]
else:
    # raw could be a single split
    split_name = list(raw.keys())[0]
    ds = raw[split_name]

print("Dataset size:", len(ds))
print("Some columns:", ds.column_names)
print("Sample record:", ds[0])


Found data files: ['/kaggle/input/finance-finetuning-dataset-combined/Finance_finetuning_dataset_combined.jsonl']


Generating train split: 0 examples [00:00, ? examples/s]

Dataset size: 20083
Some columns: ['instruction', 'input', 'output', 'question', 'answer']
Sample record: {'instruction': '<title>Minority Depository Institution (MDI: Meaning, Benefits, History</title>\n"Semiannual Report to the Congress: April 1, 2019 - September 30, 2019," Page 10.\nFederal Deposit Insurance Corporation. "2019 Minority Depository Institutions: Structure, Performance, and Social Impact," Page 55.\nFederal Deposit Insurance Corporation. "Preservation and Promotion of Minority Depository Institutions."\nFederal Reserve System. "Preserving Minority Depository Institutions - May 2020."\nFederal Register. "Minority Depository Institution Preservation Program."', 'input': 'market+news', 'output': 'The given text passages discuss Minority Depository Institutions.', 'question': None, 'answer': None}


In [8]:
import math
from transformers import PreTrainedTokenizerBase

PROMPT_TEMPLATE = (
    "### Instruction:\n{instruction}\n\n"
    "### Input:\n{input}\n\n"
    "### Response:\n"
)

def make_prompt(example):
    instruction = example.get("instruction") or example.get("Instruction") or example.get("prompt") or ""
    inp = example.get("input") or example.get("Input") or ""
    # If dataset uses 'output' or 'answer' keys:
    output = example.get("output") or example.get("Output") or example.get("answer") or example.get("Answer") or example.get("response") or ""
    prompt = PROMPT_TEMPLATE.format(instruction=instruction.strip(), input=inp.strip())
    full = prompt + output.strip()
    return prompt, full, output

def tokenize_function(example, tokenizer: PreTrainedTokenizerBase):
    prompt, full, output = make_prompt(example)
    # tokenize prompt and full separately to find token lengths
    tokenized_full = tokenizer(full, truncation=True, max_length=CUTOFF_LEN, padding=False)
    tokenized_prompt = tokenizer(prompt, truncation=True, max_length=CUTOFF_LEN, padding=False)
    input_ids = tokenized_full["input_ids"]
    prompt_len = len(tokenized_prompt["input_ids"])
    labels = input_ids.copy()
    # mask prompt tokens - so loss is computed only on the response portion
    labels[:prompt_len] = [-100] * prompt_len
    return {
        "input_ids": input_ids,
        "attention_mask": tokenized_full["attention_mask"],
        "labels": labels,
    }

# Quick tokenization test on a small sample
tok_test = tokenize_function(ds[0], tokenizer)
print("tokenized sample lengths:", {k: len(v) for k,v in tok_test.items()})


tokenized sample lengths: {'input_ids': 141, 'attention_mask': 141, 'labels': 141}


In [9]:
# Shuffle + split
ds = ds.shuffle(seed=SEED)

# small validation split: 2-5% (adjust as dataset size)
val_size = max(1, int(0.02 * len(ds)))
train_ds = ds.select(range(len(ds) - val_size))
val_ds = ds.select(range(len(ds) - val_size, len(ds)))

print("Train size:", len(train_ds), "Val size:", len(val_ds))

# Tokenize with batched mapping
def batched_tokenize(batch):
    results = {"input_ids": [], "attention_mask": [], "labels": []}
    for i in range(len(batch[next(iter(batch.keys()))])):  # iterate rows
        ex = {k: batch[k][i] for k in batch.keys()}
        tok = tokenize_function(ex, tokenizer)
        results["input_ids"].append(tok["input_ids"])
        results["attention_mask"].append(tok["attention_mask"])
        results["labels"].append(tok["labels"])
    return results

# Use batched mapping
train_tok = train_ds.map(batched_tokenize, batched=True, remove_columns=train_ds.column_names)
val_tok = val_ds.map(batched_tokenize, batched=True, remove_columns=val_ds.column_names)

print("Tokenized train columns:", train_tok.column_names)


Train size: 19682 Val size: 401


Map:   0%|          | 0/19682 [00:00<?, ? examples/s]

Map:   0%|          | 0/401 [00:00<?, ? examples/s]

Tokenized train columns: ['input_ids', 'attention_mask', 'labels']


In [10]:
from dataclasses import dataclass
from typing import Dict, List
import torch
from transformers.tokenization_utils_base import PaddingStrategy

@dataclass
class DataCollatorForCausalLM:
    tokenizer: PreTrainedTokenizerBase
    padding: bool = True
    max_length: int = CUTOFF_LEN

    def __call__(self, batch: List[Dict[str, List[int]]]) -> Dict[str, torch.Tensor]:
        input_ids = [torch.tensor(b["input_ids"], dtype=torch.long) for b in batch]
        attention_mask = [torch.tensor(b["attention_mask"], dtype=torch.long) for b in batch]
        labels = [torch.tensor(b["labels"], dtype=torch.long) for b in batch]
        # pad using tokenizer.pad
        pad_token_id = self.tokenizer.pad_token_id
        input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=pad_token_id)
        attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

data_collator = DataCollatorForCausalLM(tokenizer=tokenizer)


In [12]:
from transformers import Trainer, TrainingArguments
import inspect
import torch

# Prepare the desired kwargs (our canonical set)
desired_args = dict(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    per_device_eval_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    fp16=(True if torch.cuda.is_available() else False),
    logging_steps=50,
    evaluation_strategy="steps",  # may be accepted or not depending on transformers version
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,
    load_best_model_at_end=False,
    report_to="none",
    # ddp_find_unused_parameters only relevant in some versions; will be filtered if unsupported
    ddp_find_unused_parameters=(False if torch.cuda.is_available() else None),
)

# Inspect TrainingArguments signature and keep only supported kwargs
sig = inspect.signature(TrainingArguments.__init__)
supported_params = set(sig.parameters.keys())
# remove 'self' if present
supported_params.discard('self')

# Filter out unsupported args and None values (some versions don't accept None for ddp flag)
filtered_args = {k: v for k, v in desired_args.items() if (k in supported_params and v is not None)}

print("transformers version:", __import__("transformers").__version__)
print("TrainingArguments supports these params:", sorted(supported_params))
print("Using the following training args (filtered):")
for k, v in filtered_args.items():
    print(f"  {k}: {v}")

# Create TrainingArguments with the safe filtered args
training_args = TrainingArguments(**filtered_args)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    data_collator=data_collator,
)

print("Trainer created successfully. Starting training...")

trainer.train()

# After training, save adapters and tokenizer
print("Saving LoRA adapters and tokenizer to:", OUTPUT_DIR)
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


transformers version: 4.53.3
TrainingArguments supports these params: ['accelerator_config', 'adafactor', 'adam_beta1', 'adam_beta2', 'adam_epsilon', 'auto_find_batch_size', 'average_tokens_across_devices', 'batch_eval_metrics', 'bf16', 'bf16_full_eval', 'data_seed', 'dataloader_drop_last', 'dataloader_num_workers', 'dataloader_persistent_workers', 'dataloader_pin_memory', 'dataloader_prefetch_factor', 'ddp_backend', 'ddp_broadcast_buffers', 'ddp_bucket_cap_mb', 'ddp_find_unused_parameters', 'ddp_timeout', 'debug', 'deepspeed', 'disable_tqdm', 'do_eval', 'do_predict', 'do_train', 'eval_accumulation_steps', 'eval_delay', 'eval_do_concat_batches', 'eval_on_start', 'eval_steps', 'eval_strategy', 'eval_use_gather_object', 'fp16', 'fp16_backend', 'fp16_full_eval', 'fp16_opt_level', 'fsdp', 'fsdp_config', 'fsdp_min_num_params', 'fsdp_transformer_layer_cls_to_wrap', 'full_determinism', 'gradient_accumulation_steps', 'gradient_checkpointing', 'gradient_checkpointing_kwargs', 'greater_is_better

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
from peft import PeftModel

print("Loading base model and applying LoRA adapters for inference...")
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
)

# Load LoRA weights (this will attach them to base model)
from peft import PeftModel
lora_model = PeftModel.from_pretrained(base, OUTPUT_DIR, torch_dtype=torch.float16)
lora_model.eval()

# Example inference
def generate(prompt, max_new_tokens=256, temperature=0.2, top_p=0.95):
    inputs = tokenizer(prompt, return_tensors="pt").to(lora_model.device)
    with torch.no_grad():
        out = lora_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

# Test with a small prompt:
sample_prompt = PROMPT_TEMPLATE.format(instruction="Explain what a dividend yield is", input="")
print(generate(sample_prompt, max_new_tokens=120))
